# Split HWs

This notebook will split the HW into 3 folds for evaluation.

In [1]:
import pandas as pd
import glob
from shutil import copyfile
import os
from pathlib import Path
import unicodedata

In [2]:
# Read file from Google Drive with all the students
all_students = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vSmsR2gwRdJ8qj0lWryQgy2KINTwVq_YICiJVqLZI2WP9oNtGXKyjg05BI7H3NejOWPCIOSs3Wlflix/pub?gid=0&single=true&output=csv")
# Keep relevant columns
all_students = all_students.loc[:,['First name','Surname', 'ID number', 'Degree Code', 'Email address']]
all_students = all_students.drop_duplicates()
all_students['Full Name'] = all_students['First name'] + ' ' + all_students['Surname']
# remove apostrophe from full name
all_students['Full Name'] = all_students['Full Name'].str.replace("'", "")
# sort by full name
all_students = all_students.sort_values(by='Full Name')
all_students = all_students.reset_index(drop=True)
all_students

,First name,Surname,ID number,Degree Code,Email address,Full Name
0,AGATA,GARBIN,2072693,SC2377,agata.garbin@studenti.unipd.it,AGATA GARBIN
1,AHMAD ZUBAIR,ZAHID,2070876,SC2542,ahmadzubair.zahid@studenti.unipd.it,AHMAD ZUBAIR ZAHID
2,AIDAN SCOT,CLARK,2095143,F,aidanscot.clark@studenti.unipd.it,AIDAN SCOT CLARK
3,ALBERTO,MENGATO,2063551,SC2542,alberto.mengato@studenti.unipd.it,ALBERTO MENGATO
4,ALBERTO,ZANON,2017429,SC2542,alberto.zanon.9@studenti.unipd.it,ALBERTO ZANON
...,...,...,...,...,...,...
276,VINCENZO,SPINELLO,2097392,SC2598,vincenzo.spinello@studenti.unipd.it,VINCENZO SPINELLO
277,VINU VARSHITH,ALAGAPPAN,2041267,SC2542,vinuvarshith.alagappan@studenti.unipd.it,VINU VARSHITH ALAGAPPAN
278,VLAD-GEORGE,IONESCU,2072115,SC2542,vladgeorge.ionescu@studenti.unipd.it,VLAD-GEORGE IONESCU
279,YELNUR,SHAUKETBEK,2078709,SC2377,yelnur.shauketbek@studenti.unipd.it,YELNUR SHAUKETBEK


## [NOT WORKING] by Student ID
Next, load the folder with submissions from both channels.

In [17]:
sub_folder = Path("./HW1_1sub").resolve()
# iterate over the folders and get ID number from name
all_IDs = []
for folder in sub_folder.iterdir():
    # iterate over the files in the folder and extract ID number from filename
    for file in folder.iterdir():
        if file.is_file() and file.suffix == '.ipynb':
            all_IDs.append(file.stem.split("_")[0])
print("There are {} submisisons".format(len(all_IDs)))

There are 200 submisisons


In [24]:
# count how many elements in the list are numbers
count = 0
all_IDs_numbers = []
for i in all_IDs:
    try:
        all_IDs_numbers.append(int(i))
        count += 1
    except:
        pass

print("There are {} numbers in the list".format(count))

There are 171 numbers in the list


So 15% of students are not complying with the HW instructions!
## by Student Name

In [3]:
# Define folder with merged submissions from both channels
sub_folder = Path("./HW1_1sub").resolve()

In [4]:
# Extract Names from folders names
sub_names = []
for folder in sub_folder.iterdir():
    if folder.is_dir():
        name = folder.name.split("_")[0]
        sub_names.append(name)
        # check whether name is in all_students or not
        if name not in all_students['Full Name'].values:
            print("Name {} is not present in all_students".format(name))
print("There are {} submisisons".format(len(sub_names)))

There are 199 submisisons


In [5]:
# check names where are present in all_students
submissions = all_students[all_students['Full Name'].isin(sub_names)]
submissions

,First name,Surname,ID number,Degree Code,Email address,Full Name
0,AGATA,GARBIN,2072693,SC2377,agata.garbin@studenti.unipd.it,AGATA GARBIN
1,AHMAD ZUBAIR,ZAHID,2070876,SC2542,ahmadzubair.zahid@studenti.unipd.it,AHMAD ZUBAIR ZAHID
5,ALEJANDRA OLIVIA,CRUCES ANDREWS,2078632,SC2377,alejandraolivia.crucesandrews@studenti.unipd.it,ALEJANDRA OLIVIA CRUCES ANDREWS
7,ALESSANDRO,CANEL,2097570,SC2598,alessandro.canel@studenti.unipd.it,ALESSANDRO CANEL
8,ALESSANDRO,CORSI,2087928,SC2542,alessandro.corsi.2@studenti.unipd.it,ALESSANDRO CORSI
...,...,...,...,...,...,...
276,VINCENZO,SPINELLO,2097392,SC2598,vincenzo.spinello@studenti.unipd.it,VINCENZO SPINELLO
277,VINU VARSHITH,ALAGAPPAN,2041267,SC2542,vinuvarshith.alagappan@studenti.unipd.it,VINU VARSHITH ALAGAPPAN
278,VLAD-GEORGE,IONESCU,2072115,SC2542,vladgeorge.ionescu@studenti.unipd.it,VLAD-GEORGE IONESCU
279,YELNUR,SHAUKETBEK,2078709,SC2377,yelnur.shauketbek@studenti.unipd.it,YELNUR SHAUKETBEK


In [6]:
# Look for duplicates
dup = {x for x in sub_names if sub_names.count(x) >1}
print(dup)

set()


In [7]:
# Split the dataframe in 3 blocks
from more_itertools import divide
sub_chunks = divide(3, submissions.index)
sub_chunks = [list(x) for x in sub_chunks]
print(f"Submisisons in 3 blocks of length: {[len(sub_chunks[i]) for i in range(3)]}")

Submisisons in 3 blocks of length: [67, 66, 66]


In [8]:
# Split among TAs
TAs = ['Flavio', 'Paolo', 'Uzair']
for i, ta in enumerate(TAs):
    submissions.loc[sub_chunks[i], "TA"] = ta
submissions

C:\Users\paolo\AppData\Local\Temp\ipykernel_20684\3176314675.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submissions.loc[sub_chunks[i], "TA"] = ta


,First name,Surname,ID number,Degree Code,Email address,Full Name,TA
0,AGATA,GARBIN,2072693,SC2377,agata.garbin@studenti.unipd.it,AGATA GARBIN,Flavio
1,AHMAD ZUBAIR,ZAHID,2070876,SC2542,ahmadzubair.zahid@studenti.unipd.it,AHMAD ZUBAIR ZAHID,Flavio
5,ALEJANDRA OLIVIA,CRUCES ANDREWS,2078632,SC2377,alejandraolivia.crucesandrews@studenti.unipd.it,ALEJANDRA OLIVIA CRUCES ANDREWS,Flavio
7,ALESSANDRO,CANEL,2097570,SC2598,alessandro.canel@studenti.unipd.it,ALESSANDRO CANEL,Flavio
8,ALESSANDRO,CORSI,2087928,SC2542,alessandro.corsi.2@studenti.unipd.it,ALESSANDRO CORSI,Flavio
...,...,...,...,...,...,...,...
276,VINCENZO,SPINELLO,2097392,SC2598,vincenzo.spinello@studenti.unipd.it,VINCENZO SPINELLO,Uzair
277,VINU VARSHITH,ALAGAPPAN,2041267,SC2542,vinuvarshith.alagappan@studenti.unipd.it,VINU VARSHITH ALAGAPPAN,Uzair
278,VLAD-GEORGE,IONESCU,2072115,SC2542,vladgeorge.ionescu@studenti.unipd.it,VLAD-GEORGE IONESCU,Uzair
279,YELNUR,SHAUKETBEK,2078709,SC2377,yelnur.shauketbek@studenti.unipd.it,YELNUR SHAUKETBEK,Uzair


In [11]:
# Add columns
submissions["Final Grade"] = 0
submissions["Feedback"] = ""
# Export dataframe to excel
submissions.to_excel(sub_folder.stem + ".xlsx", index=False)

C:\Users\paolo\AppData\Local\Temp\ipykernel_20684\677610837.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submissions["Final Grade"] = 0
C:\Users\paolo\AppData\Local\Temp\ipykernel_20684\677610837.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submissions["Feedback"] = ""
